In [1]:
import os
import traceback
import logging
import os.path
import networkx as nx
import pandas as pd
import numpy as np
import pylab as pl
from matplotlib import collections  as mc
import glob
import re
import csv
import shutil

In [2]:
# simversion = '_v01'
simversion = '_plasmid_v01'
simversionPangraph = '_assemBiGraph2_v01' #'_assemGraph2_v01': use assembly graph v2; _assemBiGraph2_v01: if (u,v) in G, so is (v-, u-)
# simversion = '_plasmid_v02'
# simversion = '_plasmid_v03'
# simversion = '_plasmid_v04'
# 1 True, 0 False
run_prokka = 0
# run_panta_db = 0
prepare_database_4_multicsar = 0
run_art = 0
run_spades = 0
run_panta = 1
split_paralogs = 1
run_multicsar = 0
run_ragout = 0
run_pangraph = 1
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
pangenome_data = '/data/hoan/amromics/data/ncbi/Ecoli50random/'
prokka_target_dir = '/data/hoan/amromics/data/ncbi/Ecolitest/prokka/'
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [3]:
data_base = ' --genus Escherichia --species coli --cpus 30'
print(data_base,data_base,data_base,data_base)
print(data_base,data_base,data_base,data_base)
output_dir = pangenome_data + 'temp'
prokka_dir = pangenome_data + 'prokka'
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && '
# if run_prokka:
#     fffile = pangenome_data + 'fna/*.fna'
#     list_database = glob.glob(fffile)
#     for i in range(len(list_database)):
#         input_data = list_database[i]
#         print(input_data)
#         file_name = input_data.split("/")[-1][:-4]
#         prokka_bin = conda_dir + 'prokka --outdir ' + output_dir +data_base +' --prefix ' + file_name +' '+input_data
#         os.system(prokka_bin)
#         os.system('cp ' + output_dir + '/'+file_name+'.gff ' + prokka_dir)
#         os.system('rm -r '+ output_dir)

 --genus Escherichia --species coli --cpus 30  --genus Escherichia --species coli --cpus 30  --genus Escherichia --species coli --cpus 30  --genus Escherichia --species coli --cpus 30
 --genus Escherichia --species coli --cpus 30  --genus Escherichia --species coli --cpus 30  --genus Escherichia --species coli --cpus 30  --genus Escherichia --species coli --cpus 30


In [4]:
if prepare_database_4_multicsar==1:
    os.system('rm -r ' + pangenome_data + 'fna/')
    os.system('mkdir ' + pangenome_data + 'fna/')
    os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
    os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')

### Simulate reads using ART

In [5]:
# https://github.com/scchess/Art/blob/master/art_illumina_README

In [6]:
sim_dir = '/data/hoan/amromics/simulation/'
art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'

In [7]:
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fna'
ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002899495.1_ASM289949v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_004135915.1_ASM413591v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_006351885.1_ASM635188v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_008931765.1_ASM893176v1_genomic.fna'
# GCF_006351885_v01
sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13]
sim_output = '/data/hoan/amromics/simulation/art_output/sim_reads_' + sim_name + simversion[-4:]
# sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat_test'

In [8]:
sim_output

'/data/hoan/amromics/simulation/art_output/sim_reads_Ecolitest_GCF_002860105_v01'

In [9]:
if run_art: 
    if simversion[-3:]=='v01':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 60 -o '+sim_output) # _v01
    if simversion[-3:]=='v02':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 10 -o '+sim_output) # _v02
    if simversion[-3:]=='v03':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v03
    if simversion[-3:]=='v04':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 10 -o '+sim_output) # _v04

### Run SPADES assembly

In [10]:
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output_' + sim_name + simversion[-4:]


In [11]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)

### Run panta

In [12]:
simversionRef = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversion
simversionRef

'_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01'

In [13]:
# move spades ouput to panta dir
print('Pangenome data: ', pangenome_data)


Pangenome data:  /data/hoan/amromics/data/ncbi/Ecoli50random/


In [ ]:
if 1:
    conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && '
    input_data = spades_output + '/contigs.fasta'
    os.system('rm -r ' + prokka_target_dir +'temp/')
    prokka_bin = conda_dir + 'prokka --outdir ' + prokka_target_dir +'temp' +data_base +' --prefix ' + sim_name +' '+input_data
    os.system(prokka_bin)
    os.system('cp ' + prokka_target_dir +'temp/'+sim_name+'.gff ' + prokka_target_dir + 'g1.gff')
    os.system('cp ' + prokka_target_dir +'temp/'+sim_name+'.gff ' + prokka_target_dir + sim_name+'.gff')
    os.system('rm -r ' + prokka_target_dir +'temp/')

In [15]:
prokka_bin

'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && prokka --outdir /data/hoan/amromics/data/ncbi/Ecolitest/prokka/temp --genus Escherichia --species coli --cpus 30 --prefix Ecolitest_GCF_002860105 /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_002860105_v01/contigs.fasta'

In [16]:
# print('cp '+prokka_target_dir + sim_name+'.gff ' + pangenome_data +'prokka/')

In [17]:
pantaconda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =pantaconda_dir+'python /data/hoan/amromics/panta_memopt/pan-genome.py'
panta_output = '/data/hoan/amromics/genome-graph/scaffold_output/pantaOut/pantaOut' + simversionRef
panta_input = pangenome_data +'prokka/*.gff'
run_panta_main = 1
if run_panta_main:
    os.system('cp '+prokka_target_dir + sim_name+'.gff ' + pangenome_data +'prokka/')
    cmd_panta = panta_bin + ' main -o ' + panta_output +' -g ' + panta_input
    print(cmd_panta)
    os.system(cmd_panta)
    from pangraph.utils import export_metadata
    export_metadata(panta_output)

source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && python /data/hoan/amromics/panta_memopt/pan-genome.py main -o /data/hoan/amromics/genome-graph/scaffold_output/pantaOut/pantaOut_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01 -g /data/hoan/amromics/data/ncbi/Ecoli50random/prokka/*.gff


04:58:14 INFO : Extract protein -- time taken 0:01:29.444204
04:58:51 INFO : Run CD-HIT with 98% identity -- time taken 0:00:31.751985
05:00:18 INFO : All-against-all BLASTP -- time taken 0:01:26.463458
05:00:22 INFO : Cluster with MCL -- time taken 0:00:02.754106
05:00:22 INFO : Reinflate clusters -- time taken 0:00:00.164139
05:00:22 INFO : len inflated_clusters = 16524 len clusters = 29510
05:00:22 INFO : MEM at split_paralog0: 110.22336 0 inc = 110.22336
05:00:22 INFO : MEM at split_paralog0: 160.07168 110.22336 inc = 49.84831999999999
05:00:22 INFO : Number of clusters before spliting 16524
05:00:25 INFO : Split paralogs iterate 0 -- count = 1843 time taken 0:00:02.676036
05:00:25 INFO : MEM at split_paralog2: 171.8272 160.07168 inc = 11.755520000000018
05:00:25 INFO : Split paralogs iterate 1 -- count = 675 time taken 0:00:00.935888
05:00:25 INFO : MEM at split_paralog2: 176.90624 171.8272 inc = 5.079039999999992
05:00:26 INFO : Split paralogs iterate 2 -- count = 374 time taken 

In [18]:
# # file_name= ref_data.split("/")[-1][:-4]
# # cmd_copy = 'cp ' + prokka_target_dir + file_name + '.gff ' + prokka_target_dir + 'g1.gff'
# # panta_target_input = prokka_target_dir + file_name + '.gff'
# # os.system(cmd_copy)
# # panta_target_input = prokka_target_dir + 'g1.gff'
# ### Khong the lam nhu the nay duoc vi day co phai la cai ta can noi dau???
# # panta_target_input = prokka_target_dir + 'g1.gff'
# panta_target_input = prokka_target_dir + sim_name + '.gff'
# cmd_panta = panta_bin + ' add -c ' + panta_output +' -g ' + panta_target_input
# if run_panta:
#     os.system(cmd_panta)
#     # create samples.tsv
#     from pangraph.utils import export_metadata
#     export_metadata(panta_output)

In [19]:
os.system('rm '+ pangenome_data +'prokka/' + sim_name+'.gff')

0

In [20]:
from pangraph.utils import export_metadata
export_metadata(panta_output)

### Run pangraph

In [21]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception as e:
    logging.error(traceback.format_exc())
from pangraph import PanGraph

In [22]:
# set parameters
data_dir = panta_output 
# incomplete_sample_name = 'g1'
incomplete_sample_name = sim_name
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [23]:
simversionRefNew = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversionPangraph
# pangraph_output_greedy_default = scaffold_out_dir + 'pangraph_greedy'+ simversionRef + '.fasta'
# pangraph_output_opt_default = scaffold_out_dir + 'pangraph_opt'+ simversionRef + '.fasta'
pangraph_output_greedy = scaffold_out_dir + 'pangraph_greedy'+ simversionRefNew + '.fasta'
pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversionRefNew + '.fasta'

In [24]:
pangraph_output_opt

'/data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli50random_Ecolitest_GCF_002860105_assemBiGraph2_v01.fasta'

In [25]:
if run_pangraph:
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_greedy, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Should we use target sequence as reference, hence, the strand will be ok
Set minimum on number of nucleotides =  10 NUMBER OF COMPUTED CONTIGS: 257
Clip the matrix 0.2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Use modified  assembly graph
Use union graph: overlap + assembly
Reduce the cost by 1/2 if there is no path in the assembly graph
Reduce the cost by 1/2 if there is no path in the assembly graph
Will test this, Ok?
Success! File written


In [26]:
if run_pangraph:
    try:
        maximum_matching = 'opt'
        pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Reduce the cost by 1/2 if there is no path in the assembly graph
Reduce the cost by 1/2 if there is no path in the assembly graph
Compute maximum matching
Will test this, Ok?
Will test this, Ok?
Success! File written


In [27]:
# pangraph.adj_list0

### Run scaffold methods

In [28]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'

In [29]:
multicsar_output = scaffold_out_dir + 'multicsar/multicsar' + simversionRef
if run_multicsar:
    os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
    os.system('rm ' + pangenome_data + 'fna/g1*')
    os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
    # os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

In [30]:
multicsar_output

'/data/hoan/amromics/genome-graph/scaffold_output/multicsar/multicsar_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01'

In [31]:
### Ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'multicsar/ragout' + simversionRef
if run_ragout:
    # write receipt file
    ref_files_list = glob.glob(pangenome_data+'fna/*')
    n_files = len(ref_files_list)
    receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversionRef+'.rcp'
    f = open(receipt_file_dir, "w")
    f.write('.references = ')
    for idx in range(n_files-1):
        f.write('r' + str(idx) + ',')
    f.write('r'+str(n_files-1)+'\n')
    f.write(".target = mg1655\n\n")
    for idx in range(n_files):
        f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
    f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
    f.close()

In [32]:
if run_ragout:
    os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
    # ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

In [33]:
print(ragout_output)

/data/hoan/amromics/genome-graph/scaffold_output/multicsar/ragout_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01


### Quast 

In [34]:
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversionRefNew
quast_output

'/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_002860105_assemBiGraph2_v01'

In [35]:
quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
spades_output_fasta = spades_output+'/contigs.fasta'
multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
if 1:
    if os.path.isfile(ragout_output_fasta):
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
    else:
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

/data/hoan/amromics/spades_quast/quast-5.2.0/quast.py /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli50random_Ecolitest_GCF_002860105_assemBiGraph2_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/pangraph_greedy_Ecoli50random_Ecolitest_GCF_002860105_assemBiGraph2_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/multicsar/multicsar_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01/multi-csar.nuc.out.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_002860105_v01/contigs.fasta -l Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES -r /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna -o /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_002860105_assemBiGraph2_v01 --silent --extensive-mis-size 5000 --local-mis-size 3000


System information:
  OS: Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-redhat-7.9-Nitrogen (linux_64)
  Python version: 2.7.5
  CPUs numbe

Unable to parse the pattern
Unable to parse the pattern
Unable to parse the pattern


Done.

2023-01-18 17:03:29
Running Contig analyzer...
Done.

2023-01-18 17:03:32
Running NA-NGA calculation...
Done.

2023-01-18 17:03:33
Running Genome analyzer...
  NOTICE: No file with genomic features were provided. Use the --features option if you want to specify it.

  NOTICE: No file with operons were provided. Use the -O option if you want to specify it.
Done.

NOTICE: Genes are not predicted by default. Use --gene-finding or --glimmer option to enable it.

2023-01-18 17:03:35
Creating large visual summaries...
This may take a while: press Ctrl-C to skip this step..
  1 of 2: Creating PDF with all tables and plots...
  2 of 2: Creating Icarus viewers...
Done

2023-01-18 17:03:38
RESULTS:
  Text versions of total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_002860105_assemBiGraph2_v01/report.txt, report.tsv, and report.tex
  Text versions of transposed total report are saved to /data/hoan/amromics/genome-graph/scaf

In [36]:
# # pangraph.H["weight"][0:4]
# iteration = 0
# for u,v,a in pangraph.H.edges(data=True):
#     print(a)
#     iteration += 1
#     if iteration > 10:
#         break;

In [37]:
# nx.write_gml(pangraph.assembly_graph, quast_output+'/assembly_graph.gml')

In [38]:
pangraph.assembly_graph.number_of_edges()

3360

In [39]:
len(list(nx.weakly_connected_components(pangraph.assembly_graph)))

1

In [40]:
# pangraph.weighted_CG.nodes()

In [41]:
len(list(nx.weakly_connected_components(pangraph.H)))

240185

In [42]:
pangraph.H.number_of_nodes(), pangraph.H.number_of_edges()

(244849, 4957)

In [43]:
ref_data

'/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'

## Analysis

In [44]:
# minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli100random_Ecoli_plasmid_v01.fasta > contigs100vsref_mapping_v01a.paf
cmd_minimap = '~/miniconda3/envs/amromics/bin/minimap2 '+ref_data +' '+ spades_output +'/contigs.fasta' +' > scaffold_output/minimap_out/mapping.paf' 

In [45]:
os.system(cmd_minimap)
df = pd.read_csv("scaffold_output/minimap_out/mapping.paf", sep ="\t", header=None)

[M::mm_idx_gen::0.184*1.01] collected minimizers
[M::mm_idx_gen::0.231*1.41] sorted minimizers
[M::main::0.231*1.41] loaded/built the index for 6 target sequence(s)
[M::mm_mapopt_update::0.264*1.36] mid_occ = 16
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 6
[M::mm_idx_stat::0.285*1.33] distinct minimizers: 940802 (97.51% are singletons); average occurrences: 1.044; average spacing: 5.351; total length: 5256088
[M::worker_pipeline::0.887*2.40] mapped 421 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: /home/vanhoan310/miniconda3/envs/amromics/bin/minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_002860105_v01/contigs.fasta
[M::main] Real time: 0.900 sec; CPU: 2.143 sec; Peak RSS: 0.080 GB


In [46]:
df
ref_id = df.iloc[0,5]
ref_id

'NZ_CP019071.1'

In [47]:
final_df = df.loc[df[5] == ref_id]
final_df = final_df.sort_values(by=[7])

In [48]:
target_id = 0
gene_position = pd.read_csv(panta_output + '/gene_position.tsv', delimiter='\t', header=None)
gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']
target_contigs = [gene_position.iloc[i, 1] for i in range(len(gene_position.index)) if gene_position.iloc[i, 0]==target_id]

In [49]:
samples = pd.read_csv(panta_output + '/samples.tsv', delimiter='\t', header=None)
# samples

In [50]:
target_contigs_list = [final_df.iloc[i, 0] for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]
target_contigs_sorted = [final_df.iloc[i, 0]+'---'+str(final_df.iloc[i, 7])+'>'+str(final_df.iloc[i, 8]) for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]

In [69]:
gene_position[gene_position.ContigName=='NODE_95_length_1592_cov_58.378660'].iloc[0,2].split(';')

['Ecolitest_GCF_002860105-NODE_95_length_1592_cov_58.378660-PCILKDFA_04850@-1']

In [70]:
cluster_id = pangraph.gene2cluster_dict['Ecolitest_GCF_002860105-NODE_95_length_1592_cov_58.378660-PCILKDFA_04850@-1']

In [53]:
gene_info = pd.read_csv(panta_output + "/gene_info.tsv", delimiter='\t', header=None)
gene_info.columns =['GeneName', 'SampleID', 'clusterID']

In [71]:
gene_info[gene_info.clusterID==cluster_id]

,GeneName,SampleID,clusterID
4784,Ecolitest_GCF_002860105-NODE_95_length_1592_co...,0,4511


In [55]:
target_contigs_sorted[-4:]

['NODE_25_length_70560_cov_18.770981---4647349>4717901',
 'NODE_19_length_88449_cov_18.833710---4723262>4811707',
 'NODE_35_length_44062_cov_18.925307---4817112>4861169',
 'NODE_53_length_15724_cov_20.028017---4861615>4877333']

In [56]:
# pangraph.adj_list0

In [57]:
# cmd_dnadiff = '~/miniconda3/envs/amromics/bin/dnadiff '+ref_data +' '+ spades_output +'/contigs.fasta -p dna_diff/'+sim_name
# os.system(cmd_dnadiff)

In [58]:
# Don't use SPADES, shovil keep files.
# python readSimulator/readSimulator.py --outdir shredded_reads  --input ncbi-genomes-2023-01-07/GCF_000240185.1_ASM24018v2_genomic.fna --simulator art --simulator_path art_bin_MountRainier/art_illumina --iterations 100 --readlen 100 --depth 100  --opts '-ss HS20 -sam -p -m 400 -s 60'
# investigate why we failed? #1/ connectivity in the assembly graph:
# multiplicity of each node: now 1, change to more: use length and coverage
# assembly graph: change to 500

In [59]:
def get_node_coverage(node_id):
    return float(node_id.split('_')[5])
get_node_coverage('NODE_45_length_30843_cov_18.832500')
target_contigs_multi_1 = [item for item in target_contigs_list if get_node_coverage(item) <= 25.0]

In [60]:
def get_value(edge_df0, source_id, target_id):
    # if source_id in edge_df0['source'].values and target_id in edge_df0['target'].values:
    #     print(edge_df0[((edge_df0['source'] == source_id) & (edge_df0['target'] == target_id))])
    # else:
    #     print('No value')
    df_res = edge_df0[((edge_df0['source'] == source_id) & (edge_df0['target'] == target_id))]
    if len(df_res.index) == 1:
        print(df_res.iloc[0, 2])
    else:
        print("No value")
# pangraph.edge_df0

def next_node_multi_1(target_contigs_list, idx):
    ## find the next node of multiplicity 1 after the index: idx
    current_idx = idx + 1
    while(get_node_coverage(target_contigs_list[current_idx]) >= 25.0):
        current_idx += 1
    return target_contigs_list[current_idx]

In [61]:
# target_contigs_multi_1

In [62]:
assembly_graph = pangraph.assembly_graph
for i in range(len(target_contigs_list)-1):
    print('---------------------- i =', i, '---------------------------------------------')
    node1 = target_contigs_list[i]
    node2 = target_contigs_list[i+1]
    print(target_contigs_sorted[i],'::::::::', target_contigs_sorted[i+1])
    source_id = 'C-' + str(pangraph.tail_contig[node1])
    target_id = 'C-' + str(pangraph.head_contig[node2]) 
    source_node = node1 + pangraph.strand[node1]
    target_node = node2 + pangraph.strand[node2]
    
    if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
        print("Is there a path in the assembly graph? ", end ='')
        print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
        if nx.has_path(assembly_graph, source_node, target_node):
            path_nucleotides = -1
            path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
            for idxp in range(1, len(path)-1):
                path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
            print(' number of nucleotides: ', path_nucleotides)
        else:
            print('<----xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print("Is there a path in the pangenome graph? ", end ='')
    if nx.has_path(pangraph.H, source_id, target_id):
        p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
        weight_p = 0.0
        for node_p_idx in range(len(p)-1):
            weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
        if len(p) >= 2:
            weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
        print("Yes!, cost = ", weight_p)
    else:
        print("No path in Pangraph")
    print("Real cost between the two contigs: ", end ='')
    get_value(pangraph.edge_df0, node1, node2)
        
#     if get_node_coverage(node2) >= 25.0:
#         print('**************************************************')
#         node2 = next_node_multi_1(target_contigs_list, i+1)
#         print(node1, node2)
#         source_id = 'C-' + str(pangraph.tail_contig[node1])
#         target_id = 'C-' + str(pangraph.head_contig[node2]) 
#         source_node = node1 + pangraph.strand[node1]
#         target_node = node2 + pangraph.strand[node2]
#         get_value(pangraph.edge_df0, node1, node2)
#         if nx.has_path(pangraph.H, source_id, target_id):
#             p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
#             weight_p = 0.0
#             for node_p_idx in range(len(p)-1):
#                 weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
#             if len(p) >= 2:
#                 weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
#             print(weight_p)
#         else:
#             print("No path in Pangraph")
#         if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
#             print("Is there a path in the assembly graph? ", end ='')
#             print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
#             if nx.has_path(assembly_graph, source_node, target_node):
#                 path_nucleotides = -1
#                 path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
#                 for idxp in range(1, len(path)-1):
#                     path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
#                 print("Number of nucleo: ",  path_nucleotides) #len(path)
#             else:
#                 print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
            
    
    #     # sum_len = 0
    #     # if nx.has_path(pangraph.H, source_id, target_id) or nx.has_path(assembly_graph, source_node, target_node):
    #     #     sum_len += int(final_df.iloc[i,1])
    #     #     print(sum_len, end='')
    # else:
    #     print("Restart............................................")
    #     sum_len = 0
    # # print('')

---------------------- i = 0 ---------------------------------------------
NODE_46_length_30478_cov_18.766164---7>30478 :::::::: NODE_101_length_1318_cov_59.905780---30430>31738
Is there a path in the assembly graph? True, number of nucleotides:  -1
Is there a path in the pangenome graph? No path in Pangraph
Real cost between the two contigs: 1.25
---------------------- i = 1 ---------------------------------------------
NODE_101_length_1318_cov_59.905780---30430>31738 :::::::: NODE_42_length_35714_cov_18.958440---31691>67397
Is there a path in the assembly graph? True, number of nucleotides:  -1
Is there a path in the pangenome graph? No path in Pangraph
Real cost between the two contigs: 1.25
---------------------- i = 2 ---------------------------------------------
NODE_42_length_35714_cov_18.958440---31691>67397 :::::::: NODE_101_length_1318_cov_59.905780---67354>68662
Is there a path in the assembly graph? True, number of nucleotides:  -1
Is there a path in the pangenome graph? No

In [63]:
# pangraph.edge_df0

In [64]:
# pangraph.edge_df0['source'].values

In [65]:
## Xet chieu trong trinh dai dien de sua roi inversion.

In [66]:
# Gui 3 file reference + 1 test sample

In [67]:
# import glob
# import os
# list_file = glob.glob("/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kp50random_Kptest*/report.pdf")

In [68]:
# for file in list_file:
#     sample_id = file.split("/")[-2].split("_")[4]
#     cmd = 'mv ' + file +' '+ file[:-10] + str(sample_id) + 't3.pdf'
#     # print(cmd)
#     os.system(cmd)